# Diversification Algorithm

This notebook can take in a BM25 ranking and rerank the results for each query with our diversification algorithm. 

## Import libraries

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm


# Mount google drive to get data
(just for Marit)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive');
os.chdir('gdrive/My Drive/Information Retrieval/Information Retrieval Project/Notebooks')
os.listdir()

ModuleNotFoundError: No module named 'google'

## Functions for the algorithm

In [ ]:
def computeFairnessConstraints(G, k=10, SPC=True):
    """
    Compute the fairness constraints given the disjoint diversity groups

    Parameters
    ----------
    G:            Set of disjoint diversity groups
    k:            Number of documents in the final ranking
    SPC:          Specifies how to compute the fairness contraints
                  True  = SPC, statistical parity constraint
                  False = DIC, disparate impact constraint

    Output
    ------
    F:            Fairness constraints
    """
    g = len(G)
    F = np.zeros(g)
    if SPC:           # Statistical parity constraint
        fi = k/g
        F[:] = fi
    else:             # Disparate impact constraint
        n = sum([len(G[Gi]) for Gi in G])
        for i in range(g):
            gi = len(G[i])
            fi = k*gi/n
            F[i] = fi

    return F

def rerank(G, F, RsD, query_scores, document_id_linked_to_group, k=100, e=0.1):
    """
    Rerank the document in RsD, according to the reranking algorithm

    Parameters
    ----------
    G:            Set of disjoint diversity groups
    F:            Fairness constraints
    RsD:          Original ranking
    query_scores: BM25 scores of original ranking
    k:            Number of documents in the final ranking
    e:            Epsilon, probability to 'explore'

    Output
    ------
    reranking:    New top k ranking
    scores:       New top k scores
    """
    # Select top result of original ranking and add to Dk
    doc   = RsD[0]
    score = query_scores[0]
    Dk    = { 0: doc }
    Sk    = { 0: score }
    groupIndexOfAddedDoc = document_id_linked_to_group[doc]

    # Update currentGroupDistributionInDk
    # currentGroupDistributionInDk keeps track of how many docs of each group
    # are in the Dk
    currentGroupDistributionInDk = np.zeros(len(G))
    currentGroupDistributionInDk[groupIndexOfAddedDoc] += 1

    # Loop to add k-1 documents to Dk according to the reranking algorithm
    i = 1
    default_cluster = len(G)-1
    while i < k+1:
        p = random.random()
        if p < e:
            # Probability is smaller dan epsilon, thus randomly explore
            # j is a randomly selected group number
            j = random.randrange(len(G))    
        else:
            # Probability is bigger dan epsilon, thus exploit known information
            # Find group number which is underrepresented according to the fairness contraints
            # j is the underrepresented group number
            j = next(
                    (j for j, n in enumerate(currentGroupDistributionInDk) if n < F[j]*(i/k)),
                    default_cluster)
        # Get documents in selected group
        C = G[j]
        # Update currentGroupDistributionInDk
        currentGroupDistributionInDk[j] += 1

        # Select top result of C (according to the original ranking) and add to Dk 
        rank, doc = getTopDoc(RsD, C)
        if rank is not None and doc is not None:
            Dk[rank] = doc
            Sk[rank] = query_scores[rank]
            i = i+1
        else:
            default_cluster = default_cluster-1   
            if default_cluster is -1:
                default_cluster = len(G) - 1

    # Sort Dk and Sk according to the orginal ranking
    reranking = sorted(Dk.items())
    scores    = sorted(Sk.items())
    return [document[1] for document in reranking], [score[1] for score in scores]

def getTopDoc(RsD, D):
    """
    Get document from set D with the higest rank according to the original algorithm

    Parameters
    ----------
    D:          Group of documents
    RsD:        All documents with original ranking

    Output
    ------
    rank:       Rank of the document
    doc:        Document ID
    """
    # Loop over all documents
    for rank_index, document in enumerate(RsD):
        # If document is present in the group, return the document with rank
        if document in D:
            D.remove(document)
            return rank_index, document
    # No document present in the group
    return None, None

## Load necessary datasets in

In [ ]:
# Load in BM25 ranking
document_ranking = pd.read_csv("../Data/results/ranking.bm25.train.csv", sep=',', index_col=0, header=0 )
document_ranking.ranking = document_ranking.ranking.apply(eval)
document_ranking.scores  = document_ranking.scores.apply(eval)

# Load in LDA scores
document_lda = pd.read_csv("../Data/clean/document_lda_groups.train.csv", sep=',', index_col=0, header=0 ).fillna(0)

In [ ]:
# Preview of BM25 ranking
document_ranking.head(5)

### Create document group dictionarys

In [ ]:
def createDocumentIdLinkedToGroup(document_lda):
    document_id_linked_to_group =  {}

    # for each medical document
    for index, document_id in enumerate (document_lda.index.values):
        # Lookup the LDA group
        group = np.argmax(document_lda.iloc[index])
        # Append document ID to the dictionary under the Group key 
        document_id_linked_to_group[document_id] = group
    return document_id_linked_to_group

document_id_linked_to_group = createDocumentIdLinkedToGroup(document_lda)


In [ ]:
def createGroups(lda_scores, query_document_ranking):
    document_groups =  { 
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: [],
        12: [],
        13: []
    }

    # for each medical document
    for document_id in query_document_ranking:
        # Lookup the LDA group
        group = np.argmax(lda_scores.loc[document_id])
        # Append document ID to the dictionary under the Group key 
        document_groups[group].append(document_id)

    return document_groups

## Rerank ranking

In [ ]:
# K Number of documents in the final ranking
# e Epsilon, probability to 'explore'
def process_query(query_id, query_ranking, query_scores, lda_scores, document_id_linked_to_group, SPC = True, k = 100, e = 0.5 ):
    # Initialize/compute original ranking
    RsD = query_ranking

    # Initialize groups
    # Make set of groups
    G = createGroups(lda_scores, query_ranking)
    # Compute fairness constraints
    F = computeFairnessConstraints(G, k, SPC)

    # Rerank the original ranking with the reranking algorithm
    newRanking, newScores = rerank(G, F, RsD, query_scores, document_id_linked_to_group, k, e)

    return newRanking, newScores

### Rerank with both fairness constraints
(Both constraints seperate for only train data are at the bottom)

In [ ]:
# Specify data types
data_types = ['dev', 'test']#['train', 'dev', 'test']
# Specify the top n's
top_n = [10, 100]
# Specify the fairness constraint types
types   = ['spc', 'dic']
# Specify the epsilon values
epsilon = [0, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1]

# Loop over data types
for dt in data_types:
    print(f"\n\t    Reranking {dt} data")

    # Reading BM25 ranking in
    BM25_ranking = pd.read_csv(f"../Data/results/ranking.bm25.csv", sep=',', index_col=0, header=0 )
    BM25_ranking.ranking = BM25_ranking.ranking.apply(eval)
    BM25_ranking.scores  = BM25_ranking.scores.apply(eval)

    # Reading LDA scores
    document_lda = pd.read_csv(f"../Data/clean/document_lda_groups.csv", sep=',', index_col=0, header=0 ).fillna(0)
    document_id_linked_to_group = createDocumentIdLinkedToGroup(document_lda)

    # Loop over the top n's
    for top in top_n:
        print(f"\tReranking a new top {top}")
        # Loop over fairness constraint types
        for t in types:
            print(f"    Reranking with the {t.upper()} fairness contraint")
            # Loop over epsilon values
            for e in epsilon:
                print(f"Reranking with e={e}")

                # Initialize dataframe for reranking
                reranking_ranking_df = pd.DataFrame(columns = ['query', 'ranking', 'scores'])

                # Loop over all queries
                for query_id in tqdm(BM25_ranking.index.values):
                    # Create reranking for query
                    reranking, scores    = process_query(query_id, BM25_ranking.ranking.loc[query_id], BM25_ranking.scores.loc[query_id], document_lda, document_id_linked_to_group, e=e, k=top)
                    # Store reranking with scores
                    reranking_ranking_df = reranking_ranking_df.append({'query': query_id, 'ranking': reranking, 'scores': scores}, ignore_index=True)
                
                # Save reranking to file
                reranking_ranking_df = reranking_ranking_df.set_index('query')
                reranking_ranking_df.to_csv(f'../Data/results/{dt}/{t}/reranking.top{top}.{t}.e{e}.train.csv')


	    Reranking dev data


  0%|                                                                                         | 0/3419 [00:00<?, ?it/s]

	Reranking a new top 10
    Reranking with the SPC fairness contraint
Reranking with e=0


  3%|██▎                                                                             | 98/3419 [00:15<08:48,  6.28it/s]


KeyboardInterrupt: 

### Rerank with SPC fairness constraint

In [ ]:
Knal eruit met een error
epsilon = [0, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1]
for e in epsilon:
    reranking_ranking_df = pd.DataFrame(columns = ['query', 'ranking', 'scores'])

    for query_id in tqdm(document_ranking.index.values):
        reranking, scores    = process_query(query_id, document_ranking.ranking.loc[query_id], document_ranking.scores.loc[query_id], document_lda, document_id_linked_to_group, e=e)
        reranking_ranking_df = reranking_ranking_df.append({'query': query_id, 'ranking': reranking, 'scores': scores}, ignore_index=True)
    
    reranking_ranking_df = reranking_ranking_df.set_index('query')
    reranking_ranking_df.to_csv(f'../Data/results/spc/reranking.spc.e{e}.train.csv')

### Rerank with DIC fairness constraint

In [ ]:
Knal eruit met een error
epsilon = [0, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1]
for e in epsilon:
    reranking_ranking_df = pd.DataFrame(columns = ['query', 'ranking', 'scores'])

    for query_id in tqdm(document_ranking.index.values):
        reranking, scores    = process_query(query_id, document_ranking.ranking.loc[query_id], document_ranking.scores.loc[query_id], document_lda, document_id_linked_to_group, SPC=False, e=e)
        reranking_ranking_df = reranking_ranking_df.append({'query': query_id, 'ranking': reranking, 'scores': scores}, ignore_index=True)
        
    reranking_ranking_df = reranking_ranking_df.set_index('query')
    reranking_ranking_df.to_csv(f'../Data/results/dic/reranking.dic.e{e}.train.csv')

100%|██████████████████████████████████████████████████████████████████████████████| 3419/3419 [10:37<00:00,  5.36it/s]
